<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Convnext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 

     |████████████████████████████████| 1.9 MB 39.8 MB/s 
     |████████████████████████████████| 166 kB 70.2 MB/s 
     |████████████████████████████████| 182 kB 55.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 166 kB 68.3 MB/s 
     |████████████████████████████████| 162 kB 70.9 MB/s 
     |████████████████████████████████| 162 kB 74.7 MB/s 
     |████████████████████████████████| 158 kB 72.4 MB/s 
     |████████████████████████████████| 157 kB 75.2 MB/s 
     |████████████████████████████████| 157 kB 52.0 MB/s 
     |████████████████████████████████| 157 kB 72.6 MB/s 
     |████████████████████████████████| 157 kB 47.0 MB/s 
     |████████████████████████████████| 157 kB 68.7 MB/s 
     |████████████████████████████████| 157 kB 69.9 MB/s 
     |████████████████████████████████| 157 kB 65.8 MB/s 
     |████████████████████████████████| 156 kB 64.8 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: h

In [3]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width)  )
class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [6]:
!wget https://tfhub.dev/sayakpaul/convnext_tiny_1k_224_fe/1?tf-hub-format=compressed

--2022-11-09 17:24:31--  https://tfhub.dev/sayakpaul/convnext_tiny_1k_224_fe/1?tf-hub-format=compressed
Resolving tfhub.dev (tfhub.dev)... 142.251.12.100, 142.251.12.138, 142.251.12.139, ...
Connecting to tfhub.dev (tfhub.dev)|142.251.12.100|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/tfhub-modules/sayakpaul/convnext_tiny_1k_224_fe/1.tar.gz [following]
--2022-11-09 17:24:33--  https://storage.googleapis.com/tfhub-modules/sayakpaul/convnext_tiny_1k_224_fe/1.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103596106 (99M) [application/octet-stream]
Saving to: ‘1?tf-hub-format=compressed’

1?tf-hub-format=com 100%[===================>]  98.80M  23.7MB/s    in 5.4s    

2022-11-09 17:24:39 (18.1 MB/s) - ‘1?tf

In [7]:
from os import mkdir
mkdir("Convnext")

In [8]:
!tar -xvf "/content/1?tf-hub-format=compressed" -C "/content/Convnext" 

assets/
keras_metadata.pb
saved_model.pb
variables/
variables/variables.data-00000-of-00001
variables/variables.index


In [9]:
convnext = tf.keras.models.load_model("/content/Convnext")
model_convnext = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  convnext,
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
convnext.trainable=False
model_convnext.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 model_11 (Functional)       (None, 768)               27820128  
                                                                 
 dropout (Dropout)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 5)                 3845      
                                                                 
Total params: 27,823,973
Trainable params: 3,845
Non-trainable params: 27,820,128
_________________________________________________________________


In [10]:
model_convnext.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [11]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "convnext_tiny_1k_224",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [12]:
%%time
epochs=40
history = model_convnext.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.2850 - accuracy: 0.5014

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20221109_172549-2tfu0d07/files/model-best)... Done. 1.0s


22/22 [==============================] - 298s 9s/step - loss: 1.2850 - accuracy: 0.5014 - val_loss: 1.0434 - val_accuracy: 0.6286
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.8716 - accuracy: 0.7358

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20221109_172549-2tfu0d07/files/model-best)... Done. 1.0s


...

22/22 [==============================] - 20s 924ms/step - loss: 0.0634 - accuracy: 0.9915 - val_loss: 0.1086 - val_accuracy: 0.9714
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.0610 - accuracy: 0.9943

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20221109_172549-2tfu0d07/files/model-best)... Done. 0.9s


22/22 [==============================] - 20s 933ms/step - loss: 0.0610 - accuracy: 0.9943 - val_loss: 0.1067 - val_accuracy: 0.9714
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 0.0607 - accuracy: 0.9929

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20221109_172549-2tfu0d07/files/model-best)... Done. 1.0s


22/22 [==============================] - 20s 920ms/step - loss: 0.0607 - accuracy: 0.9929 - val_loss: 0.1059 - val_accuracy: 0.9714
CPU times: user 9min 56s, sys: 53 s, total: 10min 49s
Wall time: 21min 29s


In [13]:
wandb.finish()

accuracy,▁▄▇▇▇▇▇█████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▇▇▇▇▇▇▇███████████████████████████████
val_loss,█▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.9929
best_epoch,39
best_val_loss,0.10591
epoch,39
loss,0.06072
val_accuracy,0.97143


In [14]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
